# Chapter 50: Real-Time Stream Processing for AI
Kafka simulation, sliding windows, real-time features.


In [ ]:
import collections, random
print("Stream Processing ready")


In [ ]:
class KafkaSim:
    def __init__(self,partitions=3): self.parts={i:[] for i in range(partitions)}; self.off={i:0 for i in range(partitions)}
    def produce(self,key,value): p=hash(key)%len(self.parts); self.parts[p].append({"key":key,"value":value})
    def consume(self,n=10):
        msgs=[]
        for p,recs in self.parts.items():
            batch=recs[self.off[p]:self.off[p]+n]; self.off[p]+=len(batch); msgs.extend(batch)
        return msgs
random.seed(42); k=KafkaSim()
for i in range(20): k.produce("host-"+str(i%5),{"type":random.choice(["login","scan","transfer"]),"severity":random.randint(1,10)})
batch=k.consume(10)
print("Consumed",len(batch),"messages")
print("Sample:",batch[0])


In [ ]:
class SlidingWindow:
    def __init__(self,size=5): self.w=collections.deque(maxlen=size)
    def add(self,v): self.w.append(v)
    def mean(self): return sum(self.w)/len(self.w) if self.w else 0
sw=SlidingWindow(5)
for sev in [m["value"]["severity"] for m in batch[:8]]:
    sw.add(sev); print("window="+str(list(sw.w))+" mean="+str(round(sw.mean(),2)))
feats=collections.defaultdict(lambda:{"count":0,"sev_sum":0})
for m in batch: h=m["key"]; feats[h]["count"]+=1; feats[h]["sev_sum"]+=m["value"]["severity"]
for h,f in sorted(feats.items()): print(h+": events="+str(f["count"])+" avg_sev="+str(round(f["sev_sum"]/f["count"],1)))
